In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk

In [3]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


False

In [4]:
import re

In [5]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
# amazon=pd.read_csv("Eco_Friendly_Products_Test_Full.xlsx - Sheet1.csv")
amazon=pd.read_csv("Amazon Reviews Validation Data_1.csv")
# amazon=pd.read_csv("amazon_pos.csv")

In [7]:
amazon

,Reviews,Aspect
0,The settings are perfect for all hair types,Adaptability
1,with lots of settings,Adaptability
2,heating element has died,Durability
3,"Also, the diffuser does not stay on at all.",Ease of Use
4,and it won't stay on it at all. Just flys off....,Ease of Use
...,...,...
1176,top no longer stays on.,Performance
1177,unfortunately one of the lids does not fit at all,Performance
1178,"cheap enough that if my husband loses one, we ...",Price
1179,economical price,Price


In [8]:
# a=amazon["Review"]
a=amazon["Reviews"]

In [9]:
q=amazon["Aspect"]

In [10]:
all_values = q.str.split(',').explode()

In [11]:
unique_values = all_values.unique()

In [12]:
len_aspect=len(unique_values)

In [13]:
len_aspect

12

In [14]:
processed_series = a

In [17]:
processed_series

0             The settings are perfect for all hair types
1                                   with lots of settings
2                                heating element has died
3             Also, the diffuser does not stay on at all.
4       and it won't stay on it at all. Just flys off....
                              ...                        
1176                              top no longer stays on.
1177    unfortunately one of the lids does not fit at all
1178    cheap enough that if my husband loses one, we ...
1179                                     economical price
1180    Also, there is no way to seal the drinking hol...
Name: Reviews, Length: 1181, dtype: object

In [16]:
def extract_nouns(words):
    """Extract and return nouns from a given list of words."""
    # Join the list of words into a sentence
    # sentence = ' '.join(words)

    # Tokenize and POS tag the words in the sentence
    tokens = nltk.word_tokenize(words)
    parts_of_speech = nltk.pos_tag(tokens)

    # Extract and return words tagged as NN, NNS, NNP, or NNPS
    return [word for word, pos in parts_of_speech if pos in ['NN', 'NNS', 'NNP', 'NNPS']]


In [17]:
nouns = processed_series.apply(extract_nouns)
print(nouns)

0                                 [settings, hair, types]
1                                        [lots, settings]
2                                               [element]
3                                              [diffuser]
4                                                  [Just]
                              ...                        
1176                                              [stays]
1177                                               [lids]
1178                                            [husband]
1179                                              [price]
1180    [way, drinking, hole, car, bump, liquid, cup, ...
Name: Reviews, Length: 1181, dtype: object


In [18]:
from textblob import TextBlob

def extract_nouns_textblob(sentence):
    
    """Extract and return nouns from a given sentence using TextBlob."""
    # sentence = ' '.join(sentence)
    blob = TextBlob(sentence)
    return [word for word, tag in blob.tags if tag in ["NN", "NNS", "NNP", "NNPS"]]


In [19]:
nouns_testblob = processed_series.apply(extract_nouns_textblob)
print(nouns_testblob)

0                                 [settings, hair, types]
1                                        [lots, settings]
2                                               [element]
3                                              [diffuser]
4                                                      []
                              ...                        
1176                                              [stays]
1177                                               [lids]
1178                                            [husband]
1179                                              [price]
1180    [way, drinking, hole, car, bump, liquid, cup, ...
Name: Reviews, Length: 1181, dtype: object


In [20]:
import spacy

In [21]:
nlpt = spacy.load("en_core_web_sm")

In [22]:
def extract_nouns_spacy(sentence):
    """Extract and return nouns from a given sentence using spaCy."""
    # sentence = ' '.join(sentence)
    doc = nlpt(sentence)
    return [token.text for token in doc if token.pos_ in ["NOUN"]]

In [23]:
doc = processed_series.apply(extract_nouns_spacy)
doc

0                                 [settings, hair, types]
1                                        [lots, settings]
2                                      [heating, element]
3                                              [diffuser]
4                                                   [one]
                              ...                        
1176                                                [top]
1177                                               [lids]
1178                                            [husband]
1179                                              [price]
1180    [way, drinking, hole, car, bump, liquid, cup, ...
Name: Reviews, Length: 1181, dtype: object

In [24]:
# doc[1180]

In [25]:
def load_glove_model(glove_file):
    """Load the GloVe model from a file."""
    print("Loading GloVe Model")
    with open(glove_file, 'r') as file:
        model = {}
        for line in file:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            model[word] = embedding
        print("Done. {} words loaded!".format(len(model)))
    return model

glove_model = load_glove_model("/Users/divyamsobti/Downloads/glove.6B/glove.6B.100d.txt")


Loading GloVe Model
Done. 400000 words loaded!


# TFidf


In [26]:
def extract_nouns_tf(sentence):

    # Tokenize and POS tag the words in the sentence
    tokens = nltk.word_tokenize(sentence)
    parts_of_speech = nltk.pos_tag(tokens)
    x=[word for word, pos in parts_of_speech if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    x=" ".join(x)
    # Extract and return words tagged as NN, NNS, NNP, or NNPS
    return x

In [27]:
nouns_tf = processed_series.apply(extract_nouns_tf)

In [28]:
nouns_tf

0                                settings hair types
1                                      lots settings
2                                            element
3                                           diffuser
4                                               Just
                            ...                     
1176                                           stays
1177                                            lids
1178                                         husband
1179                                           price
1180    way drinking hole car bump liquid cup holder
Name: Reviews, Length: 1181, dtype: object

In [29]:


def extract_nouns_textblob_tf(sentence):
    
    """Extract and return nouns from a given sentence using TextBlob."""
    blob = TextBlob(sentence)
    x=[word for word, tag in blob.tags if tag in ["NN", "NNS", "NNP", "NNPS"]]
    x=" ".join(x)
    return x


In [30]:
nouns_testblob_tf = processed_series.apply(extract_nouns_textblob_tf)
print(nouns_testblob_tf)

0                                settings hair types
1                                      lots settings
2                                            element
3                                           diffuser
4                                                   
                            ...                     
1176                                           stays
1177                                            lids
1178                                         husband
1179                                           price
1180    way drinking hole car bump liquid cup holder
Name: Reviews, Length: 1181, dtype: object


In [31]:
def extract_nouns_spacy_tf(sentence):
    """Extract and return nouns from a given sentence using spaCy."""
    doc = nlpt(sentence)
    x=[token.text for token in doc if token.pos_ in ["NOUN"]]
    x=" ".join(x)
    return x

In [32]:
doc_tf = processed_series.apply(extract_nouns_spacy_tf)
doc_tf

0                                settings hair types
1                                      lots settings
2                                    heating element
3                                           diffuser
4                                                one
                            ...                     
1176                                             top
1177                                            lids
1178                                         husband
1179                                           price
1180    way drinking hole car bump liquid cup holder
Name: Reviews, Length: 1181, dtype: object

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
# Step 2: Convert text to numerical vectors using TF-IDF
vectorizer = TfidfVectorizer()
nltk = vectorizer.fit_transform(processed_series).toarray()

In [35]:
spacy_=vectorizer.fit_transform(doc_tf).toarray()

In [36]:
TextBlob_=vectorizer.fit_transform(nouns_testblob_tf).toarray()

In [37]:
def sentence_vector(word_list, model, vector_size=100):
    """Generate a sentence vector by averaging the word vectors."""
    word_vectors = [model[word] for word in word_list if word in model]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)




In [38]:
sentences_testblob=np.array(nouns_testblob)
sentences_spacy=np.array(doc)
sentences_nltk=np.array(nouns)


In [39]:
sentence_vectors_testblob = np.array([sentence_vector(sentence, glove_model) for sentence in sentences_testblob])

In [40]:
sentence_vectors_spacy = np.array([sentence_vector(sentence, glove_model) for sentence in sentences_spacy])

In [41]:
sentence_vectors_nltk = np.array([sentence_vector(sentence, glove_model) for sentence in preprocessed_ser])

# Cluster for 12 aspects

In [21]:
from sklearn.cluster import KMeans

num_clusters = len_aspect  # Example number of clusters

In [22]:
num_clusters = len_aspect  # Example number of clusters
kmeans_textblob = KMeans(n_clusters=num_clusters)
kmeans_textblob.fit(sentence_vectors_testblob)
labels_textblob = kmeans_textblob.labels_

NameError: name 'sentence_vectors_testblob' is not defined

In [44]:
kmeans_spacy = KMeans(n_clusters=num_clusters)
kmeans_spacy.fit(sentence_vectors_spacy)
labels_spacy = kmeans_spacy.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [24]:
kmeans_nltk = KMeans(n_clusters=num_clusters)
kmeans_nltk.fit(processed_series)
labels_nltk = kmeans_nltk.labels_

ValueError: could not convert string to float: 'The settings are perfect for all hair types'

# ploting

In [46]:
import plotly.express as px
import plotly.io as pio
# pio.renderers.default = "browser"

In [47]:
from sklearn.manifold import TSNE


In [48]:
import plotly.graph_objects as go
import plotly.io as pio



## textblobl

In [49]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [50]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [51]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [52]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [53]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [54]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [55]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [56]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [57]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# kmeans with 6 aspects

In [58]:
num_clusters = 6 # Example number of clusters

In [59]:
kmeans_textblob = KMeans(n_clusters=num_clusters)
kmeans_textblob.fit(sentence_vectors_testblob)
labels_textblob = kmeans_textblob.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [60]:
kmeans_spacy = KMeans(n_clusters=num_clusters)
kmeans_spacy.fit(sentence_vectors_spacy)
labels_spacy = kmeans_spacy.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [61]:
kmeans_nltk = KMeans(n_clusters=num_clusters)
kmeans_nltk.fit(sentence_vectors_nltk)
labels_nltk = kmeans_nltk.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



# ploting

## textblobl

In [62]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [63]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [64]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [65]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [66]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [67]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [68]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [69]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [70]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# GMM

In [71]:
from sklearn.mixture import GaussianMixture

In [72]:
gmm = GaussianMixture(n_components = 12)

In [73]:
gmm_6 = GaussianMixture(n_components = 6)

In [74]:
gmm.fit(sentence_vectors_nltk)
labels_nltk = gmm.predict(sentence_vectors_nltk)

In [75]:
gmm.fit(sentence_vectors_spacy)
labels_spacy = gmm.predict(sentence_vectors_spacy)

In [76]:
gmm.fit(sentence_vectors_testblob)
labels_textblob = gmm.predict(sentence_vectors_testblob)

# ploting

In [77]:
num_clusters=12

## textblobl

In [78]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [79]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [80]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [81]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [82]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [83]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [84]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [85]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [86]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# 6 cluster

In [87]:
gmm_6.fit(sentence_vectors_nltk)
labels_nltk = gmm_6.predict(sentence_vectors_nltk)

In [88]:
gmm_6.fit(sentence_vectors_spacy)
labels_spacy = gmm_6.predict(sentence_vectors_spacy)

In [89]:
gmm_6.fit(sentence_vectors_testblob )
labels_textblob = gmm_6.predict(sentence_vectors_testblob)

In [90]:
num_clusters=6

## textblobl

In [91]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [92]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [93]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [94]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [95]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [96]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [97]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [98]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [99]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# DF

In [100]:
# Create DataFrame
df_spacy = pd.DataFrame({
    'Sentence': doc,
    'Cluster': labels_spacy
})

# Show the DataFrame
print(df_spacy)

                                               Sentence  Cluster
0                               [settings, hair, types]        1
1                                      [lots, settings]        1
2                                    [heating, element]        5
3                                            [diffuser]        0
4                                                 [one]        5
...                                                 ...      ...
1176                                              [top]        5
1177                                             [lids]        3
1178                                          [husband]        5
1179                                            [price]        4
1180  [way, drinking, hole, car, bump, liquid, cup, ...        5

[1181 rows x 2 columns]


In [101]:
# Create DataFrame
df_textblob = pd.DataFrame({
    'Sentence': sentences_testblob,
    'Cluster': labels_textblob
})

# Show the DataFrame
print(df_textblob)

                                               Sentence  Cluster
0                               [settings, hair, types]        2
1                                      [lots, settings]        2
2                                             [element]        3
3                                            [diffuser]        0
4                                                    []        0
...                                                 ...      ...
1176                                            [stays]        4
1177                                             [lids]        2
1178                                          [husband]        4
1179                                            [price]        5
1180  [way, drinking, hole, car, bump, liquid, cup, ...        3

[1181 rows x 2 columns]


In [119]:
# Create DataFrame
df_nltk = pd.DataFrame({
    'Sentence': sentences_nltk,
    'Cluster': labels_nltk
})

# Show the DataFrame
print(df_nltk)

                                               Sentence  Cluster
0                               [settings, hair, types]       10
1                                      [lots, settings]        6
2                                             [element]        6
3                                            [diffuser]        0
4                                                [Just]        0
...                                                 ...      ...
1176                                            [stays]        6
1177                                             [lids]        1
1178                                          [husband]        5
1179                                            [price]        2
1180  [way, drinking, hole, car, bump, liquid, cup, ...        7

[1181 rows x 2 columns]


In [116]:
cluster_=list(set(labels_nltk))

In [117]:
feature_cluster_table = pd.DataFrame(index=unique_values, columns=cluster_)

In [118]:
feature_cluster_table

,0,1,2,3,4,5,6,7,8,9,10,11
Adaptability,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Durability,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ease of Use,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ergonomics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interference,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Performance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Use Efficiency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aesthetics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ease of Reprocessing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ease of Storage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
# add the aspect as per the sentenecs from amazon df to  df_nltk
df_=df_nltk.join(q)

In [108]:
for value in unique_values:
    for cluster in labels_nltk:
        # Count occurrences of the value in this cluster
        count = df_[(df_['Aspect'] == value) & (df_['Cluster'] == cluster)].shape[0]
        feature_cluster_table.at[value, cluster] = count

# Display the table
print(feature_cluster_table)

                       0   1   2   3  4   5
Adaptability          11   0  32  34  2   0
Durability            78  14  77  46  1   1
Ease of Use           52   0  32  13  2   1
Ergonomics            30   5  23  15  0   1
Interference          17   0  33  29  1   0
Performance           58  26  82  64  4  11
Use Efficiency        10   2  22  15  1   0
Aesthetics            45  12  45  43  0   1
Ease of Reprocessing  48   2  24  18  2   0
Ease of Storage        3   0   4   5  0   0
Price                  9   7  15   9  0  34
Safety                 1   0   6   3  0   0


In [120]:
# add the aspect as per the sentenecs from amazon df to  df_nltk
df_12=df_nltk.join(q)

In [121]:
for value in unique_values:
    for cluster in labels_nltk:
        # Count occurrences of the value in this cluster
        count = df_12[(df_['Aspect'] == value) & (df_12['Cluster'] == cluster)].shape[0]
        feature_cluster_table.at[value, cluster] = count

# Display the table
print(feature_cluster_table)

                      0   1   2  3   4   5   6   7   8   9   10  11
Adaptability           7   8   0  1   1   6  20  23   2   8   3   0
Durability            73  25   0  3   5  21  27  25   2  25   6   5
Ease of Use           49   6   0  2   1   5  21   7   3   4   2   0
Ergonomics            29   8   0  1   0   2  18   6   3   5   0   2
Interference          11   5   0  2   0   4  20  21  11   4   2   0
Performance           50  26   0  9  15  13  27  33   4  45   2  21
Use Efficiency         8   2   0  2   0  10   6   8   1  10   2   1
Aesthetics            44   5   0  1   4   3  24  16   7  15  26   1
Ease of Reprocessing  46  10   0  0   0   3  10  14   3   7   1   0
Ease of Storage        3   3   0  0   0   0   4   0   0   2   0   0
Price                  9   1  28  0   4   2  10   4   0  13   1   2
Safety                 1   0   0  0   0   0   4   3   0   2   0   0


In [122]:
glove_vectors

NameError: name 'glove_vectors' is not defined

In [ ]:
glov